# Lab 1: Build a Stateless CRM Support Agent (The Goldfish)

This lab introduces the core limitation of context-only agents.

You will build:
- A minimal LangGraph agent
- One tool: ticket lookup + ticket creation
- No memory
- A simple planner → tool → LLM loop

By the end, you will *feel* why memory is essential.


In [2]:
# !pip install langgraph langchain-openai langchain-core


In [3]:
from typing import TypedDict, Optional, Literal, Dict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
import json

llm = ChatOpenAI(model="gpt-5.1")

C:\Users\varsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ticket_db = json.load(open("data/tickets.json"))

In [5]:
@tool
def lookup_ticket(ticket_id: str):
    """Lookup a ticket by ID."""
    ticket = ticket_db.get(ticket_id)
    if ticket is None:
        return {"error": f"Ticket {ticket_id} not found."}
    return ticket


@tool
def create_ticket(customer_name: str, issue: str, device: str = "-", priority: str = "Medium"):
    """Create a new ticket."""
    new_id = str(max(int(k) for k in ticket_db.keys()) + 1)

    ticket_db[new_id] = {
        "status": "New",
        "issue": issue,
        "description": issue,
        "device": device,
        "priority": priority,
        "created_at": "2025-12-12",
        "last_updated": "2025-12-12",
        "customer_name": customer_name,
        "notes": [
            {"timestamp": "2025-12-12T13:20:58", "author": "customer", "text": issue}
        ]
    }
    return {"ticket_id": new_id, "status": "created"}


TOOLS = [lookup_ticket, create_ticket]


In [6]:
class AgentState(TypedDict):
    messages: list
    tool_result: Optional[dict]


In [7]:
def planner_node(state: AgentState):
    user_msg = state["messages"][-1].content

    planning_prompt = f"""
    You are a CRM support planner. 
    Decide whether to:
    1) call lookup_ticket
    2) call create_ticket
    3) answer directly

    User message: {user_msg}

    Return strictly JSON:
    {{
        "action": "...",
        "tool": "...",
        "arguments": {{}},
        "response": "..."
    }}
    """
    result = llm.invoke([HumanMessage(content=planning_prompt)])
    return {"messages": [AIMessage(content=result.content)]}


In [8]:
def tool_node(state: AgentState):
    raw = state["messages"][-1].content

    try:
        plan = json.loads(raw)
    except:
        return {"tool_result": {"error": "Invalid planner output"}}

    if plan.get("action") == "tool":
        tool_name = plan["tool"]
        args = plan["arguments"]

        for t in TOOLS:
            if t.name == tool_name:
                return {"tool_result": t.run(args)}

        return {"tool_result": {"error": "Unknown tool"}}

    return {"tool_result": None}


In [9]:
def response_node(state: AgentState):
    tool_result = state["tool_result"]
    user_msg = state["messages"][-1].content

    prompt = f"""
    You are a CRM agent. 
    User message: {user_msg}
    Tool result: {tool_result}

    Give a final answer to the user.
    """
    answer = llm.invoke([HumanMessage(content=prompt)])
    return {"messages": [answer]}


In [10]:
graph = StateGraph(AgentState)

graph.add_node("planner", planner_node)
graph.add_node("tool", tool_node)
graph.add_node("respond", response_node)

graph.set_entry_point("planner")
graph.add_edge("planner", "tool")
graph.add_edge("tool", "respond")
graph.add_edge("respond", END)

app = graph.compile()


In [ ]:
def pretty_print(role, text):
    print("\n" + "-" * 60)
    if role == "USER":
        print("🧑USER:")
        print(text)
    elif role == "AGENT":
        print("🤖AGENT:")
        print(text)

def chat_loop(user_text_examples):
    """
    user_text_examples: list of user queries
    """
    for user_text in user_text_examples:
        state = {"messages": [HumanMessage(content=user_text)], "tool_result": None}
        out = app.invoke(state)

        # print user and agent
        pretty_print("USER", user_text)
        reply = out["messages"][-1].content
        pretty_print("AGENT", reply)

    print("\n" + "=" * 60)



In [14]:
user_text_examples = ["My internet keeps dropping every night after 10 PM. You checked this ticket yesterday. What was the status?",
"The ticket ID is 293445. What is happening with it?",
"Okay but you already checked this a few minutes ago. Why again?",
"We already created a new ticket yesterday. Can you show it?",
"Do you remember what issue I reported earlier?",
"What was the troubleshooting advice you gave me last time?"]
chat_loop(user_text_examples)



------------------------------------------------------------
🧑USER:
My internet keeps dropping every night after 10 PM. You checked this ticket yesterday. What was the status?

------------------------------------------------------------
🤖AGENT:
I’m not finding any support tickets for your account in the last 3 days.

If you were expecting to see a recent ticket, it might be because:

- It was submitted under a different email or account.
- It was created more than 3 days ago.
- It wasn’t fully submitted (e.g., form not completed or network error).

If you tell me:
- The email you used,
- Rough date/time you contacted us,
- Channel (web form, chat, phone, etc.),
- And the issue summary,

I can help you figure out next steps or create a new ticket for you now.

------------------------------------------------------------
🧑USER:
The ticket ID is 293445. What is happening with it?

------------------------------------------------------------
🤖AGENT:
Here’s what I see for ticket **#293445